In [1]:
PROJECT_ID = 'dwh-siloam'
REGION = 'asia-southeast1'
print(f"Project ID: {PROJECT_ID}\nRegion: {REGION}")

# Initialize Vertex AI
from pathlib import Path
import vertexai
from google.cloud import aiplatform

print(f"Checking Credentials...")
if not any((Path.cwd()/"service_account").glob('*.json')):
    print("Service account folder is empty. Fallback using default gcloud account")
    aiplatform.init(project=PROJECT_ID, location=REGION)
    vertexai.init(project=PROJECT_ID, location=REGION)
else:
    print('Using service account credentials from service_account folder')
    from google.oauth2 import service_account
    sa_file = list((Path.cwd()/"service_account").glob('*.json'))[0]
    print(f"Using service account file: {sa_file}")
    credentials = service_account.Credentials.from_service_account_file(sa_file)
    aiplatform.init(project=PROJECT_ID, location=REGION, credentials=credentials)
    vertexai.init(project=PROJECT_ID, location=REGION, credentials=credentials)

# Import libraries
from langchain_google_vertexai import VertexAI, ChatVertexAI, create_structured_runnable
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

from typing import List, Optional
import requests

from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import MessagesPlaceholder
from settings import CopilotSettings
import time

Project ID: dwh-siloam
Region: asia-southeast1
Checking Credentials...
Using service account credentials from service_account folder
Using service account file: /Users/donnymirzaadhitama/workspace/others/chatbot-llm/service_account/dwh-siloam-99402e61edd2.json


In [2]:
import os
from pandasai import SmartDataframe
import pandas as pd

In [3]:
PANDASAI_API_KEY = "$2a$10$KwsNJeV.2XVPza4Ca8x9fOyMAhti4IgqBdI0vIfGg7p46b4k2EkrW"
os.environ["PANDASAI_API_KEY"] = PANDASAI_API_KEY

## Test using scratch dataframe

In [4]:
df = pd.DataFrame({
    "country": [
        "United States",
        "United Kingdom",
        "France",
        "Germany",
        "Italy",
        "Spain",
        "Canada",
        "Australia",
        "Japan",
        "China",
    ],
    "gdp": [
        19294482071552,
        2891615567872,
        2411255037952,
        3435817336832,
        1745433788416,
        1181205135360,
        1607402389504,
        1490967855104,
        4380756541440,
        14631844184064,
    ],
    "happiness_index": [6.94, 7.16, 6.66, 7.07, 6.38, 6.4, 7.23, 7.22, 5.87, 5.12],
})

df.head()

,country,gdp,happiness_index
0,United States,19294482071552,6.94
1,United Kingdom,2891615567872,7.16
2,France,2411255037952,6.66
3,Germany,3435817336832,7.07
4,Italy,1745433788416,6.38


### Test using default LLM of PandasAI

In [5]:
sdf = SmartDataframe(df)

In [6]:
response = sdf.chat("Negara mana dengan GDP tertinggi? Dan berapa GDP-nya?")
print(response)

Negara dengan GDP tertinggi adalah United States dengan GDP sebesar 19294482071552.


Proof it with pandas querying.

In [9]:
df.loc[df["gdp"] == df["gdp"].max()]

,country,gdp,happiness_index
0,United States,19294482071552,6.94


### Test using VertexAI LLM

In [11]:
config = CopilotSettings()

In [12]:
llm = VertexAI(
    model_name=config.GCP_AGENT_MODEL_NAME, 
    temperature=0, 
    max_output_tokens=8192
)

In [14]:
df = pd.read_csv("dataset/titanic/train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [15]:
sdf_vertexai = SmartDataframe("dataset/titanic/train.csv", config={"llm": llm})

In [18]:
response = sdf_vertexai.chat("Berapa rata-rata umur penumpang laki-laki yang selamat?")
print(response)

27.276021505376345


Proof it!

In [26]:
df.loc[(df["Sex"] == "male") & (df["Survived"] == 1)]["Age"].mean()

27.276021505376345

## Test using .csv file dataframe